In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import datetime
from influxdb import *

In [3]:
INFLUX_DB_IP = '10.12.97.178'
INFLUX_DB_PORT = 8086
INFLUX_DB = 'demoDb'
# READ_MEASUREMENT = 'EM_main'
# LIVE_MEASUREMENT = 'EM_live'
TAG_MEASUREMENT = 'EM_main_new'

con_obj = InfluxDBClient(host=INFLUX_DB_IP, port=INFLUX_DB_PORT, database=INFLUX_DB)
query = 'select * from ' + str(TAG_MEASUREMENT) + ' where time > now() - 1d '
df = pd.DataFrame(con_obj.query(query, chunked=True, chunk_size=10000).get_points())
df['time'] = pd.to_datetime(df['time']) 

In [4]:
df.head()

,time,DeviceID,EM_Active Power (kW),EM_Current Avg (A),EM_Current Ph1 (A),EM_Current Ph2 (A),EM_Current Ph3 (A),EM_Energy (kWh),EM_Frequency (Hz),EM_Power Factor,...,EM_Voltage Ph2-N (V),EM_Voltage Ph2-Ph3 (V),EM_Voltage Ph3-N (V),Mean_THD,imbalance_current,imbalance_volt,mean_current,mean_volt,mean_voltage,temp_rise
0,2020-05-29 14:00:51.127817984+00:00,EM22,0.0,0.0,0.0,0.0,0.0,43476.8,50.0,0.0,...,230.2,397.6,229.6,0.0,0.000000,0.201005,0.000000,398.000000,229.766667,0.080806
1,2020-05-29 14:00:51.347747072+00:00,EM5,64.3,97.1,111.9,83.7,95.8,161495.0,50.0,0.9,...,230.3,398.6,230.1,26.8,15.202471,0.016728,97.133333,398.533333,230.066667,0.000560
2,2020-05-29 14:00:51.804481024+00:00,EM9,0.0,0.0,0.0,0.0,0.0,19213.1,50.0,1.0,...,230.9,399.6,230.2,0.0,0.000000,0.225734,0.000000,398.700000,230.200000,0.101911
3,2020-05-29 14:00:51.820108032+00:00,EM10,0.0,0.0,0.0,0.0,0.0,124157.2,50.0,0.0,...,240.8,416.2,239.4,0.0,0.000000,0.456841,0.000000,415.900000,240.133333,0.417407
4,2020-05-29 14:00:51.977788928+00:00,EM23,0.0,0.0,0.0,0.0,0.0,71110.1,50.0,0.0,...,239.2,415.5,240.2,0.0,0.000000,0.040090,0.000000,415.733333,240.000000,0.003214


In [ ]:

class Frequency():
    
    def __init__(self):
        self.DFDBClient = DataFrameClient(host=cg.INFLUX_DB_IP, port=cg.INFLUX_DB_PORT, database=cg.INFLUX_DB)
    
    def read_Data(self):
        con_obj = InfluxDBClient(host=cg.INFLUX_DB_IP, port=cg.INFLUX_DB_PORT, database=cg.INFLUX_DB)
        query = 'select * from ' + cg.TARGET_MEASUREMENT  + ' where time > now() - 1d '
        self.df = pd.DataFrame(con_obj.query(query, chunked=True, chunk_size=10000).get_points())
        self.df['time'] = pd.to_datetime(self.df['time'])
        self.df['time'] = self.df['time'] + datetime.timedelta(hours=5, minutes=30)
        return self.df
     
    def time_as_index(self,df):
        t = pd.DataFrame(df.groupby(['DeviceID'])['time'].max())
        t.reset_index(inplace = True)
        return t
    
    def time(self,df):
        df['time'] = pd.to_datetime(df['time'])
        df['Time_max'] = df['time'].dt.time
        a=df.iloc[list(df.groupby('DeviceID')['EM_Frequency (Hz)'].idxmax())][['Time_max','DeviceID']]
        b=df.iloc[list(df.groupby('DeviceID')['EM_Frequency (Hz)'].idxmin())][['Time_max','DeviceID']]
        a.index=a['DeviceID']
        a=a.drop('DeviceID',axis=1)
        a['Time_min']=list(b['Time_max'])
        return a
    
    def categorization_count(self,df):
        bins = [0, 49, 51, np.inf]
        names = ['Count_low_freq', 'Count_normal_freq', 'Count_high_freq']
        d = dict(enumerate(names, 1))
        df['Status'] = np.vectorize(d.get)(np.digitize(df['EM_Frequency (Hz)'], bins))
        return df
    
    def calculate_count(self,df): 
        df=self.categorization_count(df)
        c = pd.DataFrame(df.groupby(['DeviceID', 'Status']).size()).reset_index()
        c = pd.pivot_table(index ='DeviceID', columns ='Status', values = 0, data=c,aggfunc=np.sum)
        c.reset_index(inplace=True)
        return c
    
    def categorization_time(self,df):
        bins = [0, 49, 51, np.inf]
        names = ['Time_low_freq', 'Time_normal_freq', 'Time_high_freq']
        d = dict(enumerate(names, 1))
        df['Status'] = np.vectorize(d.get)(np.digitize(df['EM_Frequency (Hz)'], bins))
        return df
    
    def calculate_time(self,df): 
        x=pd.DataFrame(df.groupby([(df.Status != df.Status.shift()).cumsum()])['time'].apply(lambda x:(x.iloc[-1]-x.iloc[0]).total_seconds()/60))
        x['Status']=df.loc[df.Status.shift(-1) != df.Status]['Status'].values
        x.reset_index(drop=True,inplace=True)
        return x
    
    def output(self):
        df = self.read_Data()
        x = df.groupby('DeviceID')['EM_Frequency (Hz)'].describe()
        x.columns = ['Total_Count', 'Average', 'SD', 'Minimum', '25th_percentile', 'Median', '75th_percentile', 'Maximum']
        t = self.time_as_index(df)
        x=x.merge(t,on='DeviceID')
        a=self.time(df)
        x=x.merge(a,on='DeviceID')
        c = self.calculate_count(df)
        x=x.merge(c,on='DeviceID')
        df=self.categorization_time(df)
        y=df.groupby('DeviceID').apply(self.calculate_time)
        y=y.reset_index()
        y=pd.pivot_table(index ='DeviceID', columns ='Status', values = 'time',data=y,aggfunc=np.sum) 
        y.reset_index(inplace=True)  
        x=x.merge(y,on='DeviceID')
        x = x.fillna(0)
        x.set_index('time', inplace = True)
        print(self.DFDBClient.write_points(x, cg.FREQUENCY))
        return x        
    


In [10]:
cat = Frequency()
t=cat.output(df)
t

,DeviceID,Total_Count,Average,SD,Minimum,25th_percentile,Median,75th_percentile,Maximum,Time_max,Time_min,Count_low_freq,Count_normal_freq,Time_low_freq,Time_normal_freq
time,,,,,,,,,,,,,,,
2020-05-25 07:35:29.013076992,EM1,33460.0,49.934758,0.063829,49.0,49.9,49.9,50.0,50.1,13:00:40.999438,15:37:43.661748,0.0,33460.0,0.0,1423.130691
2020-05-25 07:35:29.562092032,EM10,32471.0,50.020320,0.040238,50.0,50.0,50.0,50.0,50.1,07:52:32.487720,07:52:19.648230,0.0,32471.0,0.0,1423.165231
2020-05-25 07:35:29.434907904,EM2,34853.0,49.944257,0.062944,49.0,49.9,49.9,50.0,50.1,08:03:22.176659,06:48:03.122767,0.0,34853.0,0.0,1423.135638
2020-05-25 07:35:28.841207040,EM22,32521.0,50.000058,0.002416,50.0,50.0,50.0,50.0,50.1,08:25:46.171396,07:52:19.741977,0.0,32521.0,0.0,1423.151654
2020-05-25 07:35:28.513027840,EM23,32530.0,50.019843,0.039883,50.0,50.0,50.0,50.0,50.1,07:52:26.795439,07:52:21.594636,0.0,32530.0,0.0,1423.115307
2020-05-25 07:35:29.825526016,EM3,27880.0,49.951180,0.062643,49.0,49.9,50.0,50.0,50.2,13:31:25.233789,15:37:43.458627,0.0,27880.0,0.0,1423.153718
2020-05-25 07:35:27.122353920,EM4,27539.0,49.999568,0.038644,44.0,50.0,50.0,50.0,50.0,07:52:19.585732,00:31:29.546057,2.0,27537.0,0.0,1422.933966
2020-05-25 07:35:29.966155008,EM5,33687.0,49.998709,0.045707,45.4,50.0,50.0,50.0,50.0,07:52:21.891505,00:11:12.902808,3.0,33684.0,0.0,1422.880308
2020-05-25 07:35:28.934953984,EM6,32884.0,50.000000,0.000000,50.0,50.0,50.0,50.0,50.0,07:52:19.679478,07:52:19.679478,0.0,32884.0,0.0,1423.154258


In [5]:
bins = [0, 49, 51, np.inf]
names = ['Count_low_freq', 'Count_normal_freq', 'Count_high_freq']
d = dict(enumerate(names, 1))
df['Status'] = np.vectorize(d.get)(np.digitize(df['EM_Frequency (Hz)'], bins))
df.head()

,time,DeviceID,EM_Active Power (kW),EM_Current Avg (A),EM_Current Ph1 (A),EM_Current Ph2 (A),EM_Current Ph3 (A),EM_Energy (kWh),EM_Frequency (Hz),EM_Power Factor,...,EM_Voltage Ph2-Ph3 (V),EM_Voltage Ph3-N (V),Mean_THD,imbalance_current,imbalance_volt,mean_current,mean_volt,mean_voltage,temp_rise,Status
0,2020-05-29 14:00:51.127817984+00:00,EM22,0.0,0.0,0.0,0.0,0.0,43476.8,50.0,0.0,...,397.6,229.6,0.0,0.000000,0.201005,0.000000,398.000000,229.766667,0.080806,Count_normal_freq
1,2020-05-29 14:00:51.347747072+00:00,EM5,64.3,97.1,111.9,83.7,95.8,161495.0,50.0,0.9,...,398.6,230.1,26.8,15.202471,0.016728,97.133333,398.533333,230.066667,0.000560,Count_normal_freq
2,2020-05-29 14:00:51.804481024+00:00,EM9,0.0,0.0,0.0,0.0,0.0,19213.1,50.0,1.0,...,399.6,230.2,0.0,0.000000,0.225734,0.000000,398.700000,230.200000,0.101911,Count_normal_freq
3,2020-05-29 14:00:51.820108032+00:00,EM10,0.0,0.0,0.0,0.0,0.0,124157.2,50.0,0.0,...,416.2,239.4,0.0,0.000000,0.456841,0.000000,415.900000,240.133333,0.417407,Count_normal_freq
4,2020-05-29 14:00:51.977788928+00:00,EM23,0.0,0.0,0.0,0.0,0.0,71110.1,50.0,0.0,...,415.5,240.2,0.0,0.000000,0.040090,0.000000,415.733333,240.000000,0.003214,Count_normal_freq


In [9]:
c = pd.DataFrame(df.groupby(['DeviceID', 'Status']).size()).reset_index()
c = pd.pivot_table(index ='DeviceID', columns ='Status', values = 0, data=c,aggfunc=np.sum)
c.reset_index(inplace=True)
c

Status,DeviceID,Count_low_freq,Count_normal_freq
0,EM1,2883.0,9913.0
1,EM10,NaN,14510.0
2,EM22,NaN,14551.0
3,EM23,NaN,14543.0
4,EM4,NaN,12099.0
5,EM5,NaN,14605.0
6,EM6,NaN,14692.0
7,EM7,NaN,15539.0
8,EM9,NaN,13668.0
